In [1]:
# Tu będę importował potrzebne biblioteki
import pandas as pd
import numpy as np

#### Case study

Dane: HappyCustomerBank
Opis problemu i danych:

About Company

Happy Customer Bank is a mid-sized private bank which deals in all kinds of loans. They have presence across all major cities in India and focus on lending products. They have a digital arm which sources customers from the internet.

Problem

Digital arms of banks today face challenges with lead conversion, they source leads through mediums like search, display, email campaigns and via affiliate partners. Here Happy Customer Bank faces same challenge of low conversion ratio. They have given a problem to identify the customers segments having higher conversion ratio for a specific loan product so that they can specifically target these customers, here they have provided a partial data set for salaried customers only from the last 3 months. They also capture basic details about customers like gender, DOB, existing EMI, employer Name, Loan Amount Required, Monthly Income, City, Interaction data and many others. Let’s look at the process at Happy Customer Bank.

In above process, customer applications can drop majorly at two stages, at login and approval/ rejection by bank. Here we need to identify the segment of customers having higher disbursal rate in next 30 days.

Data Set

We have train and test data set, train data set has both input and output variable(s). Need to predict probability of disbursal for test data set.

Input variables:

    ID - Unique ID (can not be used for predictions)
    Gender- Sex
    City - Current City
    Monthly_Income - Monthly Income in rupees
    DOB - Date of Birth
    Lead_Creation_Date - Lead Created on date
    Loan_Amount_Applied - Loan Amount Requested (INR)
    Loan_Tenure_Applied - Loan Tenure Requested (in years)
    Existing_EMI - EMI of Existing Loans (INR)
    Employer_Name - Employer Name
    Salary_Account- Salary account with Bank
    Mobile_Verified - Mobile Verified (Y/N)
    Var5- Continuous classified variable
    Var1- Categorical variable with multiple levels
    Loan_Amount_Submitted- Loan Amount Revised and Selected after seeing Eligibility
    Loan_Tenure_Submitted- Loan Tenure Revised and Selected after seeing Eligibility (Years)
    Interest_Rate- Interest Rate of Submitted Loan Amount
    Processing_Fee- Processing Fee of Submitted Loan Amount (INR)
    EMI_Loan_Submitted- EMI of Submitted Loan Amount (INR)
    Filled_Form- Filled Application form post quote
    Device_Type- Device from which application was made (Browser/ Mobile)
    Var2- Categorical Variable with multiple Levels
    Source- Categorical Variable with multiple Levels
    Var4- Categorical Variable with multiple Levels

Outcomes:

    LoggedIn- Application Logged (Variable for understanding the problem – cannot be used in prediction)
    Disbursed- Loan Disbursed (Target Variable)

Źródło:

https://discuss.analyticsvidhya.com/t/hackathon-3-x-predict-customer-worth-for-happy-customer-bank/3802
Interesują nas dwie miary jakości rozwiązania:

    AUC

    Zysk, liczony w następujący sposób:
        zaklasyfikowanie obserwacji jako 1 kosztuje nas 100 zł (tzn. nasz model przypisuje obserwacji klasę 1),
        trafienie predykcją w klasę 1 przynosi nam 1000 zł zarobku.

Cel: osiągnąć jak największy zysk.


## propozycja kolejności pracy :
1. wczytanie danych 
2. podejrzenie danych 
3. wybranie co jest y(Target Variable) i sprawdzenie co to za dane (what object it is)
4. wybranie kolumn, które nie będą potrzebne i wyrzucenie ich:
    - sprawdzenie co to za obiekty 
    - i ile jest unikalnych obiektów 
5. Wymyślenie co zrobić z nanami (jak są reprezentowane braki) i jak je uzupełnić (0 , średnia(rozkład gaussa), mediana(rozkład skośny), stworzyć nowy label, lub czy warto wyrzucić (ale raczej nie robimy tego)
 !!!#ZACZYNAM OD TEGO W Środę
6. Wymyślenie co z zrobić z miastami (pogrupować jak w poprzednim zadaniu)
7. Na tym co zostanie zrobić one-hot encoder aby mieć gotową dataframe do modelowania
8. Podzielić dane na zbiór testowy i treningowy :
    - czy występuje problem niezbalansowanych klas jak sobie z nim poradzić 
9. Modelowanie (wybieranie najlepszych modeli na podstawie zarobek = zysk - koszt i/lub AUC)
10. Wybieranie odpowiednich parametrów 
11. Wybranie najlepszego modelu
12. Modelowanie na danych z rozwiązanym problemem niezbalansowanych klas i pogrupowaną kolumną city
13. wybranie najlepszego modelu.


Work Content:
1. Loading the File
2. Looking at data:
    - Checking what objects are there
    - How many unique objects are there
    - Check how many missing values are there and how are they represented in data.
3. Choosing which column is y (Target Variable):
    - checking what object it is
    - checking what distribution it has 
    - figuring out imbalanced data problem 
4. Choosing columns which I don't need in it and deleting them.
5. Discovering what to do with missing data.
6. Choosing what i am doing with City feature and how to represent this feature in data
7. Making one-hot encoder to be dataframe ready to modeling.
8. Spliting data to train and test data. 
9. Choosing classifiers to model data 
10. Hyperparameter-optimization:
    - RandomizedSearchCV 
    - Tree-structured Parzen Estimator
11. Choosing the best model 
12. Model data with imblalance class problem solved and with regroup city column.
13. Choosing best model.


### 1. Wczytanie danych 
Przy wczytywaniu danych należy zwrócić uwagę na odpowiednie dobranie parametrów (hyperparam). W naszym przypadku ważne jest odpowiednie kodowanie(latin1)- należało sprawdzić w pliku jak jest on zakodowany 
Plik czytamy za pomocą pd.read_csv (zamienia pliki csv na DataFrame) 
Następnie patrzymy na pierwsze 5 wierszy aby dowiedzieć jak wygląda nasza ramka danych.

1. Loading the File
In this case i need to pay attention to choose good encoding parameter in read_csv function. Becouse this file is coded as latin1. Also this function reads this file as dataframe

In [2]:
train_data = pd.read_csv('C:\\Users\\piotr\\Data Science bootcamp ML\dane_3_4\Dataset\Train_nyOWmfK.csv', encoding='latin1') #
test_data = pd.read_csv('C:\\Users\\piotr\\Data Science bootcamp ML\dane_3_4\Dataset\Test_bCtAN1w.csv', encoding='latin1' )
# looking at 5 first rows of data
train_data.head()
# train data - dane z Target Variable, na których będziemy uczyć i testować dane 

# test_data - dane, na których sprawdzany będzie model bez Target Variable 

,ID,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,...,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,LoggedIn,Disbursed
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000.0,5.0,0.0,CYBOSOL,...,NaN,NaN,NaN,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),...,13.25,NaN,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,...,NaN,NaN,NaN,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000.0,5.0,0.0,BIHAR GOVERNMENT,...,NaN,NaN,NaN,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000,17-Feb-84,20-May-15,500000.0,2.0,25000.0,GLOBAL EDGE SOFTWARE,...,NaN,NaN,NaN,N,Web-browser,B,S134,3,1,0


In [3]:
test_data.head()

,ID,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,...,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4
0,ID000026A10,Male,Dehradun,21500,03-Apr-87,05-May-15,100000.0,3.0,0.0,APTARA INC,...,100000.0,3.0,20.0,1000.0,2649.39,N,Web-browser,B,S122,3
1,ID000054C40,Male,Mumbai,42000,12-May-80,01-May-15,0.0,0.0,0.0,ATUL LTD,...,690000.0,5.0,24.0,13800.0,19849.90,Y,Mobile,C,S133,5
2,ID000066O10,Female,Jaipur,10000,19-Sep-89,01-May-15,300000.0,2.0,0.0,SHAREKHAN PVT LTD,...,NaN,NaN,NaN,NaN,NaN,N,Web-browser,B,S133,1
3,ID000110G00,Female,Chennai,14650,15-Aug-91,01-May-15,0.0,0.0,0.0,MAERSK GLOBAL SERVICE CENTRES,...,NaN,NaN,NaN,NaN,NaN,N,Mobile,C,S133,1
4,ID000113J30,Male,Chennai,23400,22-Jul-87,01-May-15,100000.0,1.0,5000.0,SCHAWK,...,100000.0,2.0,NaN,NaN,NaN,N,Web-browser,B,S143,1


###  wnioski
- train_data zawiera kolumnę Disbursed(Target Variable)- będzie ona y 
- test_data nie zawiera Disbursed(Target Variable) (W naszym problemie te dane są zbędne ( nie mamy jak sprawdzić czy zrobiliśmy dobrą predykcję)
- kolumny do wyrzucenia na pierwszym miejscu:
    -LoggedIn (wiemy o ty z treści zadania)
    -ID (Id klienta w banku)
- z kolumn DOB i Lead_Creation_Date wyciągnę wiek każdego klienta


## conclusions
- train_data has Disbursed column to be Target Variable - y 
- test_data doesn't contain the Disbursed column (in our case this will be unnecessary dataset)(I can't check if i have good model on this dataset
- First, I will delete the Loggedin nad ID columns
- Then, I will pull age of clients from columns: DOB and Lead_Creation_Date

### 2.Podejrzenie i opisanie danych 
- train_data.describe: Robimy tabelkę gdzie opisaną mamy każdą kolumnę(cechę)(feature) dzięki czemu możemy przeanalizować unikalną liczbę wartości w każdej kolumnie, zobaczyć gdzie brakuje danych( pod warunkiem, że są to nan lub None(chyba).

- train_data.dtypes: Sprawdzamy jaki jest są typy obiektów w danych (co musimy zamienić tak, żeby mieć float lub integer w każdej kolumnie

- train_data.isnull().sum(): sprawdzamy w jakich kolumnach i ile mamy wartość nan(brak danych)

Przy preprocessingu danych bardzo ważnym zadaniem jest zrozumienie danych, aby mieć odpowiednie podejście do radzenia sobie z brakiem danych( missing data) (ale nie tylko). W tym przypadku głównie będę zamieniał nan(missing data) na 0.   

### 2. Checking and describing data 
- function describes making a table which shows count and unique values in all columns 
- function dtypes shows which objects are in which columns
- Function isnull.sum shows sum of missing values in each column

There is a difficult problem in preprocessing data to understand business in the dataset. This is important to manage those missing values. In this dataset i will be replacing missing values to 0 value and to strings. 

In [4]:
# describes dataset 
train_data.describe(include = 'all')

,ID,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,...,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,LoggedIn,Disbursed
count,87020,87020,86017,8.702000e+04,87020,87020,8.694900e+04,86949.000000,8.694900e+04,86949,...,27726.000000,27420.000000,27726.000000,87020,87020,87020,87020,87020.000000,87020.000000,87020.000000
unique,87020,2,697,NaN,11345,92,NaN,NaN,NaN,43567,...,NaN,NaN,NaN,2,2,7,30,NaN,NaN,NaN
top,ID113213J30,Male,Delhi,NaN,11-Nov-80,03-Jul-15,NaN,NaN,NaN,0,...,NaN,NaN,NaN,N,Web-browser,B,S122,NaN,NaN,NaN
freq,1,49848,12527,NaN,306,2315,NaN,NaN,NaN,4914,...,NaN,NaN,NaN,67530,64316,37280,38567,NaN,NaN,NaN
mean,NaN,NaN,NaN,5.884997e+04,NaN,NaN,2.302507e+05,2.131399,3.696228e+03,NaN,...,19.197474,5131.150839,10999.528377,NaN,NaN,NaN,NaN,2.949805,0.029350,0.014629
std,NaN,NaN,NaN,2.177511e+06,NaN,NaN,3.542068e+05,2.014193,3.981021e+04,NaN,...,5.834213,4725.837644,7512.323050,NaN,NaN,NaN,NaN,1.697720,0.168785,0.120062
min,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,0.000000,0.000000e+00,NaN,...,11.990000,200.000000,1176.410000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,1.650000e+04,NaN,NaN,0.000000e+00,0.000000,0.000000e+00,NaN,...,15.250000,2000.000000,6491.600000,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000
50%,NaN,NaN,NaN,2.500000e+04,NaN,NaN,1.000000e+05,2.000000,0.000000e+00,NaN,...,18.000000,4000.000000,9392.970000,NaN,NaN,NaN,NaN,3.000000,0.000000,0.000000
75%,NaN,NaN,NaN,4.000000e+04,NaN,NaN,3.000000e+05,4.000000,3.500000e+03,NaN,...,20.000000,6250.000000,12919.040000,NaN,NaN,NaN,NaN,5.000000,0.000000,0.000000


### wnioski
Zauważam, że kolumna Employer_name ma 43567 unikalnych wartości na 86567 wszystkich, na tej podstawie odrzucam tę kolumnę. 
Możemy zastanowić się nad pogrupowaniem tej kolumny pod względem czy jakaś firma jest państwowa czy nie ( ale jest to bardzo dużo pracy niekoniecznie sensownej)
Mamy 697 miast(City) (tu grupowanie na regiony może mieć sens dlatego, spróbuję to zrobić).


### Conclusion 
I noticed that the Employer_name column has 43567 unique values. I also discovered that this column contains names of companies.
I am wondering, whether i should group this column by private or government sectors, but i think this is unncessary.
I also think to regroup the city column in some way for it to make sense. (I will work on it at end of my work)

In [5]:
# dtyptes function 
train_data.dtypes

ID                        object
Gender                    object
City                      object
Monthly_Income             int64
DOB                       object
Lead_Creation_Date        object
Loan_Amount_Applied      float64
Loan_Tenure_Applied      float64
Existing_EMI             float64
Employer_Name             object
Salary_Account            object
Mobile_Verified           object
Var5                       int64
Var1                      object
Loan_Amount_Submitted    float64
Loan_Tenure_Submitted    float64
Interest_Rate            float64
Processing_Fee           float64
EMI_Loan_Submitted       float64
Filled_Form               object
Device_Type               object
Var2                      object
Source                    object
Var4                       int64
LoggedIn                   int64
Disbursed                  int64
dtype: object

### wnioski
Jest dużo kolumn typu object, które będe zamieniał na integer lub float
### Conclusion 
There are a lot of object type columns, which i need to change to either integer or float

In [6]:
# sprawdzamy w jakich kolumnach i ile mamy wartość nan(brak danych)
# checking in which columns there are missing values 
train_data.isnull().sum()

ID                           0
Gender                       0
City                      1003
Monthly_Income               0
DOB                          0
Lead_Creation_Date           0
Loan_Amount_Applied         71
Loan_Tenure_Applied         71
Existing_EMI                71
Employer_Name               71
Salary_Account           11764
Mobile_Verified              0
Var5                         0
Var1                         0
Loan_Amount_Submitted    34613
Loan_Tenure_Submitted    34613
Interest_Rate            59294
Processing_Fee           59600
EMI_Loan_Submitted       59294
Filled_Form                  0
Device_Type                  0
Var2                         0
Source                       0
Var4                         0
LoggedIn                     0
Disbursed                    0
dtype: int64

In [7]:
# 5 first rows from Salary_Account column.
train_data['Salary_Account'].head(10)

0              HDFC Bank
1             ICICI Bank
2    State Bank of India
3    State Bank of India
4              HDFC Bank
5                   HSBC
6               Yes Bank
7                    NaN
8    State Bank of India
9             Kotak Bank
Name: Salary_Account, dtype: object

### Wnioski 
W kolumnie Salary_Account zawierającej nazwy banków, brakujące dane zamienię na 'no bank'
W kolumnie City brakujące dane zamienię na 'no city'
W pozostałych kolumnach gdzie są brakujące dane zamienię je na 0 (oprócz Employer Name, którą usunę)


### Conclusion 
In the Salary_Account column with names of banks, i will change missing data to 'no bank' string. 
In City column i will change missing data to 'no city' - and later i will regroup this column to 'no city', 'big city', 'small city'.
In each column, i need to change missing values to '0'.

### 3. Wybranie Target variable
Z treści zadania wynika, że Disbursed to y
Sprawdzam liczności 0 i 1, żeby dowiedzieć się czy klasy będą nierównomierne

### 3. Choosing Target Variable- y
From the content of the instruction, i know to choose Disbursed column as target variable. In order to count of 0 and 1 in this column to deduce that this feature has imbalanced class.

In [8]:
licznosc_1 = np.where(train_data['Disbursed'] == 1)[0].shape[0]
licznosc_0 = np.where(train_data['Disbursed'] == 0)[0].shape[0]
(licznosc_1/(licznosc_0+licznosc_1))*100

1.462882096069869

In [9]:

y = train_data.Disbursed
X = train_data.drop(['Disbursed'], axis = 1)

In [10]:
X

,ID,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,...,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,LoggedIn
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000.0,5.0,0.0,CYBOSOL,...,NaN,NaN,NaN,NaN,N,Web-browser,G,S122,1,0
1,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),...,2.0,13.25,NaN,6762.90,N,Web-browser,G,S122,3,0
2,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,...,4.0,NaN,NaN,NaN,N,Web-browser,B,S143,1,0
3,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000.0,5.0,0.0,BIHAR GOVERNMENT,...,5.0,NaN,NaN,NaN,N,Web-browser,B,S143,3,0
4,ID000009J40,Male,Bengaluru,100000,17-Feb-84,20-May-15,500000.0,2.0,25000.0,GLOBAL EDGE SOFTWARE,...,2.0,NaN,NaN,NaN,N,Web-browser,B,S134,3,1
5,ID000010K00,Male,Bengaluru,45000,21-Apr-82,20-May-15,300000.0,5.0,15000.0,COGNIZANT TECHNOLOGY SOLUTIONS INDIA PVT LTD,...,5.0,13.99,1500.0,6978.92,N,Web-browser,B,S143,3,1
6,ID000011L10,Female,Sindhudurg,70000,23-Oct-87,01-May-15,6.0,5.0,0.0,CARNIVAL CRUISE LINE,...,NaN,NaN,NaN,NaN,N,Web-browser,B,S133,1,0
7,ID000012M20,Male,Bengaluru,20000,25-Jul-75,20-May-15,200000.0,5.0,2597.0,GOLDEN TULIP FLORITECH PVT. LTD,...,5.0,NaN,NaN,NaN,N,Web-browser,B,S159,3,0
8,ID000013N30,Male,Kochi,75000,26-Jan-72,02-May-15,0.0,0.0,0.0,SIIS PVT LTD,...,5.0,14.85,26000.0,30824.65,Y,Mobile,C,S122,5,0
9,ID000014O40,Female,Mumbai,30000,12-Sep-89,03-May-15,300000.0,3.0,0.0,SOUNDCLOUD.COM,...,3.0,18.25,1500.0,10883.38,N,Web-browser,B,S133,1,0


### Wnioski
Liczność 1 wynosi 1.5 % (czyli występuje problem nierównomierności klas)
"Zaproponować jakieś rozwiązanie problemu"

### Conclusion 
Counts 1 is 1.5% and 0 is 98.5%. I assume this data has imbalanced class problem beacouse of that distribution. (i will solve this problem later in this notebook)

### 4. Usunięcie kolumn
Usuwam kolumny:
    - Employer Name 
    - LoggedIn (wiemy o ty z treści zadania)
    - ID (Id klienta w banku)
    - po wyciągnięciu wieku usunę DOB i Lead_Creation_Date

### 5. Columns Removal
I am removing columns:
    - Employer Name 
    - LoggedIn (i know to remove this column from the content)
    - ID (customers ID)
    - i will take age from DOB and Lead_Creation_Date. Then i will remove this column

In [11]:
X = X.drop(['Employer_Name', 'LoggedIn', 'ID'],axis = 1) 


stworzenie kolumny age: 115(dane dla klientów z 2015) - dwia ostanie znaki z kolumny(DOB) data urodzenia(rok) 

I am creating new column age. I took age from two last marks from DOB column and substract this from 115.

In [12]:
X['age'] = [115-int(s[-2:]) for s in X.DOB]
X = X.drop(['DOB', 'Lead_Creation_Date'],axis = 1)

In [13]:
X

,Gender,City,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Salary_Account,Mobile_Verified,Var5,Var1,...,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,age
0,Female,Delhi,20000,300000.0,5.0,0.0,HDFC Bank,N,0,HBXX,...,NaN,NaN,NaN,NaN,N,Web-browser,G,S122,1,37
1,Male,Mumbai,35000,200000.0,2.0,0.0,ICICI Bank,Y,13,HBXA,...,2.0,13.25,NaN,6762.90,N,Web-browser,G,S122,3,30
2,Male,Panchkula,22500,600000.0,4.0,0.0,State Bank of India,Y,0,HBXX,...,4.0,NaN,NaN,NaN,N,Web-browser,B,S143,1,34
3,Male,Saharsa,35000,1000000.0,5.0,0.0,State Bank of India,Y,10,HBXX,...,5.0,NaN,NaN,NaN,N,Web-browser,B,S143,3,28
4,Male,Bengaluru,100000,500000.0,2.0,25000.0,HDFC Bank,Y,17,HBXX,...,2.0,NaN,NaN,NaN,N,Web-browser,B,S134,3,31
5,Male,Bengaluru,45000,300000.0,5.0,15000.0,HSBC,Y,17,HAXM,...,5.0,13.99,1500.0,6978.92,N,Web-browser,B,S143,3,33
6,Female,Sindhudurg,70000,6.0,5.0,0.0,Yes Bank,N,0,HBXX,...,NaN,NaN,NaN,NaN,N,Web-browser,B,S133,1,28
7,Male,Bengaluru,20000,200000.0,5.0,2597.0,NaN,Y,3,HBXX,...,5.0,NaN,NaN,NaN,N,Web-browser,B,S159,3,40
8,Male,Kochi,75000,0.0,0.0,0.0,State Bank of India,Y,13,HAXB,...,5.0,14.85,26000.0,30824.65,Y,Mobile,C,S122,5,43
9,Female,Mumbai,30000,300000.0,3.0,0.0,Kotak Bank,Y,0,HBXC,...,3.0,18.25,1500.0,10883.38,N,Web-browser,B,S133,1,26


## 5. Uzupełnienie brakujących danych 
W następujących kolumnach zamieniam brakujące dane (nan) na 0 :
Loan_Amount_Submitted, Loan_Tenure_Submitted, Interest_Rate, Processing_Fee, EMI_Loan_Submitted, Existing_EMI, Loan_Tenure_Applied, Loan_Amount_Applied.
W kolumnie Salary_Account braki danych zamieniam na 'no bank', w kolumnie City na 'no City'

### 5. Filling missing data
In the following columns, i am replacing missing data to '0': Loan_Amount_Submitted, Loan_Tenure_Submitted, Interest_Rate, Processing_Fee, EMI_Loan_Submitted, Existing_EMI, Loan_Tenure_Applied, Loan_Amount_Applied. 
In Salary_Account missing data i am replacing it to 'no bank' string and in city column to 'no city'.

In [14]:
X['Loan_Amount_Submitted'] = X['Loan_Amount_Submitted'].fillna(0)
X['Loan_Tenure_Submitted'] = X['Loan_Tenure_Submitted'].fillna(0)
X['Interest_Rate'] = X['Interest_Rate'].fillna(0)
X['Processing_Fee'] = X['Processing_Fee'].fillna(0)
X['EMI_Loan_Submitted'] = X['EMI_Loan_Submitted'].fillna(0)
X['Loan_Tenure_Applied'] = X['Loan_Tenure_Applied'].fillna(0)
X['Loan_Amount_Applied'] = X['Loan_Amount_Applied'].fillna(0)
X['Existing_EMI'] = X['Existing_EMI'].fillna(0)

In [15]:
# sprawdzamy w jakich kolumnach i ile mamy wartość nan(brak danych)
X.isnull().sum()

Gender                       0
City                      1003
Monthly_Income               0
Loan_Amount_Applied          0
Loan_Tenure_Applied          0
Existing_EMI                 0
Salary_Account           11764
Mobile_Verified              0
Var5                         0
Var1                         0
Loan_Amount_Submitted        0
Loan_Tenure_Submitted        0
Interest_Rate                0
Processing_Fee               0
EMI_Loan_Submitted           0
Filled_Form                  0
Device_Type                  0
Var2                         0
Source                       0
Var4                         0
age                          0
dtype: int64

In [16]:
X['Salary_Account'] = X['Salary_Account'].fillna('no bank')
X['City'] = X['City'].fillna('no city')

In [17]:
X.isnull().sum()

Gender                   0
City                     0
Monthly_Income           0
Loan_Amount_Applied      0
Loan_Tenure_Applied      0
Existing_EMI             0
Salary_Account           0
Mobile_Verified          0
Var5                     0
Var1                     0
Loan_Amount_Submitted    0
Loan_Tenure_Submitted    0
Interest_Rate            0
Processing_Fee           0
EMI_Loan_Submitted       0
Filled_Form              0
Device_Type              0
Var2                     0
Source                   0
Var4                     0
age                      0
dtype: int64

### Wnioski
 Moim zdaniem wyrzuciłem nie potrzebne kolumny, mam dane bez brakujących wartości. 



### Conclusion 
I removed unnecessary columns, therefore i received all columns without missing values.


### 6. Co zrobić z kolumną miasta

Kolejnym etapem jest pogrupowanie np. od liczby mieszkańców lub stanu. 

### 6. City column
Next step is to label columns by number of customers in each city. I would group this column by number of clients who applied for loans from diffrent cities. According to me, the size of the city of origin (client) might influence the results. I will work on this theory later. 

In [18]:
X.head()

,Gender,City,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Salary_Account,Mobile_Verified,Var5,Var1,...,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,age
0,Female,Delhi,20000,300000.0,5.0,0.0,HDFC Bank,N,0,HBXX,...,0.0,0.00,0.0,0.0,N,Web-browser,G,S122,1,37
1,Male,Mumbai,35000,200000.0,2.0,0.0,ICICI Bank,Y,13,HBXA,...,2.0,13.25,0.0,6762.9,N,Web-browser,G,S122,3,30
2,Male,Panchkula,22500,600000.0,4.0,0.0,State Bank of India,Y,0,HBXX,...,4.0,0.00,0.0,0.0,N,Web-browser,B,S143,1,34
3,Male,Saharsa,35000,1000000.0,5.0,0.0,State Bank of India,Y,10,HBXX,...,5.0,0.00,0.0,0.0,N,Web-browser,B,S143,3,28
4,Male,Bengaluru,100000,500000.0,2.0,25000.0,HDFC Bank,Y,17,HBXX,...,2.0,0.00,0.0,0.0,N,Web-browser,B,S134,3,31


In [21]:
X.groupby(['City']).count().loc[['no city']]

,Gender,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Salary_Account,Mobile_Verified,Var5,Var1,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,age
City,,,,,,,,,,,,,,,,,,,,
no city,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003


In [104]:
import matplotlib.pyplot as plt
%matplotlib inline


For now i deleted City column and i will fit classifier without this column

In [26]:
X1 = X.drop(['City'],axis = 1) 


In [62]:
X1

,Gender,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Salary_Account,Mobile_Verified,Var5,Var1,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,age
0,Female,20000,300000.0,5.0,0.0,HDFC Bank,N,0,HBXX,0.0,0.0,0.00,0.0,0.00,N,Web-browser,G,S122,1,37
1,Male,35000,200000.0,2.0,0.0,ICICI Bank,Y,13,HBXA,200000.0,2.0,13.25,0.0,6762.90,N,Web-browser,G,S122,3,30
2,Male,22500,600000.0,4.0,0.0,State Bank of India,Y,0,HBXX,450000.0,4.0,0.00,0.0,0.00,N,Web-browser,B,S143,1,34
3,Male,35000,1000000.0,5.0,0.0,State Bank of India,Y,10,HBXX,920000.0,5.0,0.00,0.0,0.00,N,Web-browser,B,S143,3,28
4,Male,100000,500000.0,2.0,25000.0,HDFC Bank,Y,17,HBXX,500000.0,2.0,0.00,0.0,0.00,N,Web-browser,B,S134,3,31
5,Male,45000,300000.0,5.0,15000.0,HSBC,Y,17,HAXM,300000.0,5.0,13.99,1500.0,6978.92,N,Web-browser,B,S143,3,33
6,Female,70000,6.0,5.0,0.0,Yes Bank,N,0,HBXX,0.0,0.0,0.00,0.0,0.00,N,Web-browser,B,S133,1,28
7,Male,20000,200000.0,5.0,2597.0,no bank,Y,3,HBXX,200000.0,5.0,0.00,0.0,0.00,N,Web-browser,B,S159,3,40
8,Male,75000,0.0,0.0,0.0,State Bank of India,Y,13,HAXB,1300000.0,5.0,14.85,26000.0,30824.65,Y,Mobile,C,S122,5,43
9,Female,30000,300000.0,3.0,0.0,Kotak Bank,Y,0,HBXC,300000.0,3.0,18.25,1500.0,10883.38,N,Web-browser,B,S133,1,26


### 7. Zamieniamy wszystkie dane typu objcect, one-hot encoder

### 7. Replacing the data type object with one-hot encoder
In order to do this, i will use the get_dummies function

W tym celu używamy funkcji get_dummies 

In [28]:
X1 = pd.get_dummies(X1, drop_first = True)

In [29]:
# dane przygotowane do modelowania
X1

,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Var5,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,...,Source_S153,Source_S154,Source_S155,Source_S156,Source_S157,Source_S158,Source_S159,Source_S160,Source_S161,Source_S162
0,20000,300000.0,5.0,0.0,0,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
1,35000,200000.0,2.0,0.0,13,200000.0,2.0,13.25,0.0,6762.90,...,0,0,0,0,0,0,0,0,0,0
2,22500,600000.0,4.0,0.0,0,450000.0,4.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
3,35000,1000000.0,5.0,0.0,10,920000.0,5.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
4,100000,500000.0,2.0,25000.0,17,500000.0,2.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
5,45000,300000.0,5.0,15000.0,17,300000.0,5.0,13.99,1500.0,6978.92,...,0,0,0,0,0,0,0,0,0,0
6,70000,6.0,5.0,0.0,0,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
7,20000,200000.0,5.0,2597.0,3,200000.0,5.0,0.00,0.0,0.00,...,0,0,0,0,0,0,1,0,0,0
8,75000,0.0,0.0,0.0,13,1300000.0,5.0,14.85,26000.0,30824.65,...,0,0,0,0,0,0,0,0,0,0
9,30000,300000.0,3.0,0.0,0,300000.0,3.0,18.25,1500.0,10883.38,...,0,0,0,0,0,0,0,0,0,0


### 8. Dzielimy dane na treningowe i testowe 
Ilość danych testowych wynosi 20000, random_state służy do ustalenia ziarna losowości 

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [25]:
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size = 20000 ,random_state = 123)

### 9. Modelling (choosing best models based on earnings = profit - expense or/and on AUC)
I will fit the model on the training data. In this case, i am interested with parameters earnings=profit-expense and AUC, not the accuracy itself, since it is inbalanced data. In the next step, i will find the best models with the best hyperparameters and compare diffrent results, with AUC or earnings. 

### 9. Modelowanie (wybieranie najlepszych modeli na podstawie zarobek = zysk - koszt i/lub AUC)
Fitujemy model na danych treningowych i testowych.  W tym przypadku nie interesuje nas accuracy_score ponieważ dane są niezbalansowane i accuracy wyjdzie bardzo wysokie nawet przy najgorszym modelu.( Pytanie czy coś z tym zrobić).
W dalszym w etapie poszukamy odpowiednich parametrów dla modelu, a także spróbujemy porównać różne modele, tak aby zysk wyszedł jak największy.


In [ ]:
Dobranie odpowiedniej liczby obserwacji w liściu (mała), deep= -1 lub none żeby drzewo doszło do odpowiedniej głębokości,
i dobrze ustawić próg klasyfikacji ( mniejszy niż 0.5 - standardowo ustawiony). 
To są wskazówki jak dobrać paramety random forest dla niezbalansowanej liczności klas bez samlingu lub dodawania danych